<a href="https://colab.research.google.com/github/manikantmnnit/RAG/blob/main/Hybrid_Search_and_reranking_in_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install weaviate-client install langchain langchain-community

In [33]:
import weaviate
import os

In [34]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN_Key')

In [35]:
auth_config = weaviate.AuthApiKey(api_key="QzZCaKS9jW0laKzJYkMe9ZpnGrdbdwUFYfEa")

client = weaviate.Client(
  url="https://hybrid-search-ztcug8ad.weaviate.network",
  auth_client_secret=auth_config,
  additional_headers = {
        "X-HuggingFace-Api-Key": HF_TOKEN
        }
)

In [36]:
client.is_ready()


True

In [37]:
# The database schema defines how data is stored, organized and retrieved in Weaviate
schema = {
    "classes": [
        {
            "class": "RAG",
            "description": "Documents for RAG",
            "vectorizer": "text2vec-huggingface",
            "moduleConfig": {"text2vec-huggingface": {"model": "sentence-transformers/all-MiniLM-L6-v2", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-huggingface": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}


In [38]:
client.schema.create(schema)

UnexpectedStatusCodeError: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name RAG already exists'}]}.

In [39]:
client.schema.get()

{'classes': [{'class': 'RAG',
   'description': 'Documents for RAG',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the paragraph',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-huggingface': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Wed Jul  3 18:34:58 2024",
     'indexFilterable': True,
     'indexSearchabl

In [40]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

In [41]:
retriever = WeaviateHybridSearchRetriever(
    alpha = 0.5,               # defaults to 0.5, which is equal weighting between keyword and semantic search
    client = client,           # keyword arguments to pass to the Weaviate client
    index_name = "RAG",  # The name of the index to use
    text_key = "content",         # The name of the text key to use
    attributes = [], # The attributes to return in the results
    create_schema_if_missing=True,
)

In [42]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [43]:
!pip install bitsandbytes accelerate

In [44]:
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

In [45]:
# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

In [46]:
# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [47]:
tokenizer = initialize_tokenizer(model_name)

In [48]:
model = load_quantized_model(model_name)

Unused kwargs: ['low_cpu_mem_usage']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [49]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=3000,
    do_sample=True,
    top_k=5,
    max_new_tokens=100,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [50]:
llm = HuggingFacePipeline(pipeline=pipeline)
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ff5d2bb8190>)

In [51]:
doc_path="/content/CSSP-D-24-00691_reviewer_2.pdf"

In [52]:
!pip install pypdf

In [53]:
from langchain_community.document_loaders import PyPDFLoader

In [54]:
loader = PyPDFLoader(doc_path)

In [55]:
docs = loader.load()

In [56]:
docs

[Document(metadata={'source': '/content/CSSP-D-24-00691_reviewer_2.pdf', 'page': 0}, page_content='Circuits, Systems, and Signal Processing  \nCriteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters \n--Manuscript Draft-- \n \nManuscript Number: CSSP-D-24-00691\nFull Title: Criteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters\nArticle Type: Original Research\nKeywords: Asymptotic stability;  Digital filters;  External disturbance;  Generalized overflow\nnonlinearities;  Very strict passivity\nAbstract: This brief proposes novel asymptotic stability criteria for nonlinear digital filters\nsubjected to external disturbances and overflow nonlinearities. The proposed\ncriteria, formulated within the linear matrix inequality framework, guarantees\nthe passive behavior of the nonlinear digital filter based on its storage function.\nThese conditions are established via a very strict passivity approach, making\nthem applicable to

In [57]:
len(docs)

16

In [73]:
retriever.add_documents(docs)

['38ca3334-140f-4569-b7dc-dfbdc7966f2f',
 'fdc611fb-9944-412f-a37a-49719d017397',
 '5ba2780a-29e0-4407-aeb9-04da991ac45d',
 '218280d0-d49e-47c3-9aae-ddace648b710',
 'a8e73f8a-13b9-4ac9-a68b-604889b4bc2a',
 'b79546ac-8f3c-4849-881c-7ccadfa91ce3',
 '9077749f-7043-4b3c-98de-d7fd1e8cfa24',
 '1884e9dd-0197-49ee-9e9f-e7fd08b48820',
 '202b5f2e-6723-4ecb-9cf9-99f4605dc137',
 'a1d8872e-c66f-43ea-8866-c4d84c066ff8',
 '70f0a112-b1f7-488f-95db-0dcd51afb6ab',
 '1cef5cff-162f-4bf0-b0d5-ef028dbead56',
 'd124445c-f5ba-4583-859f-75753f3fc19a',
 '16ac7733-19e1-4ff7-937f-243fa14e5d93',
 '13e31de7-a8eb-47c5-b708-d147ae8218be',
 'b6bcaf2b-1957-4314-917c-6655b3cb8bc7']

In [96]:
QA1=retriever.invoke("what is digital filters?",)


In [97]:
print(QA1)
print(QA1[1])

[Document(page_content='Circuits, Systems, and Signal Processing  \nCriteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters \n--Manuscript Draft-- \n \nManuscript Number: CSSP-D-24-00691\nFull Title: Criteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters\nArticle Type: Original Research\nKeywords: Asymptotic stability;  Digital filters;  External disturbance;  Generalized overflow\nnonlinearities;  Very strict passivity\nAbstract: This brief proposes novel asymptotic stability criteria for nonlinear digital filters\nsubjected to external disturbances and overflow nonlinearities. The proposed\ncriteria, formulated within the linear matrix inequality framework, guarantees\nthe passive behavior of the nonlinear digital filter based on its storage function.\nThese conditions are established via a very strict passivity approach, making\nthem applicable to various nonlinearity types, including zeroing, saturation, two’s\ncompleme

In [98]:
retriever.invoke(
    "what is RAG token?",
    score=True
)

[Document(metadata={'_additional': {'explainScore': '\nHybrid (Result Set vector,hybridVector) Document b6bcaf2b-1957-4314-917c-6655b3cb8bc7: original score 0.45863342, normalized score: 0.5', 'score': '0.5'}}, page_content='  Cover Letter\nClick here to access/download \nElectronic Supplementary Material \nCover Letter.pdf '),
 Document(metadata={'_additional': {'explainScore': '\nHybrid (Result Set vector,hybridVector) Document d7681a52-de5b-4d9f-a72f-542c8c5d7797: original score 0.45863342, normalized score: 0.5', 'score': '0.5'}}, page_content='  Cover Letter\nClick here to access/download \nElectronic Supplementary Material \nCover Letter.pdf '),
 Document(metadata={'_additional': {'explainScore': '\nHybrid (Result Set vector,hybridVector) Document 5f64db22-785a-45f2-83c6-d81e0f78e0e9: original score 0.25414497, normalized score: 0.28060973', 'score': '0.28060973'}}, page_content='Theorem 1 Suppose if there exist real matrices P=PT>0,S=ST>0and\nscalars ϵ0>0,γ >0such that\nϑ1=\uf8e

In [99]:
from langchain.chains import RetrievalQA

In [100]:
from langchain_core.prompts import ChatPromptTemplate

In [63]:
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

In [64]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)

In [65]:
from langchain.prompts import PromptTemplate
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

In [66]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [67]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [101]:
hybrid_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,)


In [109]:
result1 = hybrid_chain.invoke("what is digital filters?")
print(result1)

Both `max_new_tokens` (=100) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'query': 'what is digital filters?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nCircuits, Systems, and Signal Processing  \nCriteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters \n--Manuscript Draft-- \n \nManuscript Number: CSSP-D-24-00691\nFull Title: Criteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters\nArticle Type: Original Research\nKeywords: Asymptotic stability;  Digital filters;  External disturbance;  Generalized overflow\nnonlinearities;  Very strict passivity\nAbstract: This brief proposes novel asymptotic stability criteria for nonlinear digital filters\nsubjected to external disturbances and overflow nonlinearities. The proposed\ncriteria, formulated within the linear matrix inequality framework, guarantees\nthe passive behavior of the nonlinear digital filter based on its 

In [110]:
print(result1['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Circuits, Systems, and Signal Processing  
Criteria to Mitigate Overflow Oscillations in Externally Interfered Digital Filters 
--Manuscript Draft-- 
 
Manuscript Number: CSSP-D-24-00691
Full Title: Criteria to Mitigate Overflow Oscillations in Externally Interfered Digital Filters
Article Type: Original Research
Keywords: Asymptotic stability;  Digital filters;  External disturbance;  Generalized overflow
nonlinearities;  Very strict passivity
Abstract: This brief proposes novel asymptotic stability criteria for nonlinear digital filters
subjected to external disturbances and overflow nonlinearities. The proposed
criteria, formulated within the linear matrix inequality framework, guarantees
the passive behavior of the nonlinear digital filter based on its storage function.
These conditions are established via a very strict 

In [111]:
query="what is external interference in digital filter?"

In [112]:
response = hybrid_chain.invoke({"query":query})

Both `max_new_tokens` (=100) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [125]:
print(response['result']['Helpful Answer'])

TypeError: string indices must be integers

In [132]:
print(response['query'])
print(response['result'][-550:])

what is external interference in digital filter?
swer: In the context of digital filters, external interference refers to unwanted or disturbing signals that are introduced into the system from outside sources, such as noise or other signals that may affect the accuracy and stability of the filter. These external disturbances can cause oscillations, instability, and other issues in the filter's performance, and it is important to develop criteria and methods to mitigate the effects of external interference and ensure the filter's stability and accuracy. The proposed criteria in this brief aim


In [133]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [134]:
# Set up the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    prompt |
    llm
)

In [135]:
query="what is digital filters?"

In [136]:
response=rag_chain.invoke(query)

Both `max_new_tokens` (=100) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [137]:
print(response)


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
[Document(page_content='Circuits, Systems, and Signal Processing  \nCriteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters \n--Manuscript Draft-- \n \nManuscript Number: CSSP-D-24-00691\nFull Title: Criteria to Mitigate Overflow Oscillations in\xa0Externally Interfered Digital Filters\nArticle Type: Original Research\nKeywords: Asymptotic stability;  Digital filters;  External distu

In [ ]:
response

In [139]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [140]:
!pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 11.2 MB/s eta 0:00:00


In [142]:
compressor = CohereRerank(cohere_api_key="XaM8Iw3XZYrTyjgw1MdMBNekmc2kQh59aMRSdBov")

In [144]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
    )

In [146]:
compressed_docs = compression_retriever.get_relevant_documents(query)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'relevance_score': 0.79549927}, page_content='Criteria to Mitigate Overflow Oscillations in\nExternally Interfered Digital Filters\nMounika Pulikonda and Priyanka Kokil\nAdvanced Signal and Image Processing (ASIP) Lab, Department of\nElectronics and Communication Engineering, Indian Institute of\nInformation Technology, Design and Manufacturing, Kancheepuram,\nChennai, 600127, India.\n*Corresponding author(s). E-mail(s): priyanka@iiitdm.ac.in\nContributing authors: ec21d0001@iiitdm.ac.in\nAbstract\nThis brief proposes novel asymptotic stability criteria for nonlinear digital fil-\nters subjected to external disturbances and overflow nonlinearities. The proposed\ncriteria, formulated within the linear matrix inequality framework, guarantees\nthe passive behavior of the nonlinear digital filter based on its storage function.\nThese conditions are established via a very strict passivity approach, making\nthem applicable to various nonlinearity types, including zeroing,

In [164]:

for doc in compressed_docs:
  print(doc.metadata['relevance_score'])

0.79549927
0.79549927
0.53776973


In [147]:
hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=compression_retriever
)

In [148]:
response = hybrid_chain.invoke("What is external interference?")

Both `max_new_tokens` (=100) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
print(response.get("result"))

In [ ]:
print(response.get("result"))